### Sampled plots for ideal seperation of controls

In [ ]:
### Sampled-control plots based on r value, p value, and y value

import plotly.plotly as py
import plotly
import pandas as pd
import seaborn
import glob
import numpy
from scipy import stats
import re
import matplotlib.pyplot as plt
from plotly.offline import iplot, init_notebook_mode
init_notebook_mode()

# Check in with plotly
plotly.tools.set_credentials_file(username='joch5340', api_key='JjrFk5oHORN8g2N9hWIG')

# Enter save dir here
out_dir = '/home/archie/code/DnA/out/presy/'

# nix that pesky # of figures error
plt.rcParams.update({'figure.max_open_warning': 0})

# Enter data directories here

dir_home= '/home/archie/code/DnA/data/'
dir_list= ['promoters', 'repressors', 'control']

# Create dataframe for lin-reg stats
df2 = pd.DataFrame(columns=['mode', 'Gene', 'slope','intercept', 'r_value', 'p_value', 'std_err', 'spearman', 'comb','sort', 'mean_MD'])

# Main
for directory in dir_list:
    print (dir_home + directory)
    for filepath in glob.iglob(dir_home + directory + "/HO*.tsv"):
  # Parse
        fields = ['non-promoterbi_MD', 'max_exp_of_TF', 'numtotalbi', 'SRR', 'max_exp_isoform' ]
        df = pd.read_csv(filepath, sep='\t', usecols=fields)
        np = 0
        nm = 0
        mean_MD = round(df['non-promoterbi_MD'].mean(), 5)
        for i in range(0, (len(df['non-promoterbi_MD']))):
            if (df.loc[i, 'numtotalbi'] < 5000):
                df = df.drop([i])
            try: 
                if (df.loc[i, 'non-promoterbi_MD'] > 0.1):
                    np +=1
                elif (df.loc[i, 'non-promoterbi_MD'] < 0.1):
                    nm +=1
            except:
                print ("")
        pop = ((len(df['non-promoterbi_MD']))*0.75)
        if np >= pop or mean_MD >= .115:
            sort = 'promoter'
        elif nm >= pop:
            sort ='repressor'
        else:
            sort = 'uncertain'

    # Get Gene Title
        regexp = "\/(?:HO_)(.*_.*)\."
        Title = re.findall(regexp, filepath)[0]
        regexp1 = "\/(?:HO_)(.*)(?:_.*)\." 
        regexp2 = "\/(?:HO_)(?:.*)(?:_.*)(.[A-Z])\."
        r1 = re.findall(regexp1, filepath)[0]
        r2 = re.findall(regexp2, filepath)[0]
        Gene = (r1 +  r2)
   # Generate lin-reg with sci-py
        slope, intercept, r_value, p_value, std_err = stats.linregress(df['max_exp_of_TF'], df['non-promoterbi_MD'])
        mode = directory
        y_value = df['non-promoterbi_MD'].corr(df['max_exp_of_TF'], method='spearman')
        comb = ((y_value + r_value)/2)
        df2.loc[Title] = [mode, Gene, slope, intercept, r_value, p_value, std_err, y_value, comb, sort, mean_MD]

# Brace yourself, interactive plotlies are coming
      
print ("\nr_value vs p_value")
fig = {
    'data': [
            {
             'x': df2['r_value'], 
            'y': df2['p_value'], 
            'text': "mode : " + df2['mode'] + " Gene : " + df2['Gene'],
            'mode': 'markers'},
            ], 
            }

#py.iplot(fig, filename='multiple-scatter')


#iplot(fig)

print ("\nspearmans ro vs p_value")
fig = {
    'data': [
            {
             'x': df2['spearman'], 
            'y': df2['p_value'], 
            'text': "mode : " + df2['mode'] + " Gene : " + df2['Gene'],
            'mode': 'markers'},
            ], 
            }

py.iplot(fig, filename='multiple-scatter')


iplot(fig)

fig = {
    'data': [
            {
             'x': df2['r_value'], 
            'y': df2['p_value'], 
            'text':"mode : " +  df2['mode'] + " Gene : " + df2['Gene'],
            'mode': 'markers'},
            ], 
            }

py.iplot(fig, filename='multiple-scatter')


iplot(fig)

df2['x'] = ''
#seaborn.violinplot(y='non-promoterbi_MD', x='tenths', data=df, orient='v', bw=.2, scale="width")

plt.figure()
seaborn.lmplot(x="comb", y="slope", data=df2, hue='mode', fit_reg=False)
plt.savefig(out_dir + Title + "combvslope.png")

plt.figure()
seaborn.lmplot(x="r_value", y="p_value", data=df2, hue='mode', fit_reg=False)
plt.savefig(out_dir + Title + "rvp.png")


plt.figure()
seaborn.lmplot(x="spearman", y="p_value", data=df2, hue='mode', fit_reg=False)
plt.savefig(out_dir + Title + "yvp.png")


plt.figure()
seaborn.lmplot(x="comb", y="p_value", data=df2, hue='mode', fit_reg=False)
plt.savefig(out_dir + Title + "combvp.png")


plt.figure()
seaborn.lmplot(x="r_value", y="spearman", data=df2, hue='mode', fit_reg=False)
plt.figure()
fig1 = seaborn.violinplot(y='p_value', x='x', data=df2, orient='v', hue='mode', \
                          inner='stick', bw=.3, scale="width")
plt.ylim(-.1,.25)
plt.show()
plt.figure()
fig1 = seaborn.violinplot(y='r_value', x='x', data=df2, orient='v',  hue='mode', \
                          inner='stick', bw=.3, scale="area")
plt.show()
plt.figure()
fig1 = seaborn.violinplot(y='slope', x='x', data=df2, orient='v', hue='mode', \
                          inner='stick', bw=.3, scale="area")
plt.show()
plt.figure()
fig1 = seaborn.violinplot(y='intercept', x='x', data=df2, orient='v',  hue='mode', \
                          inner='stick', bw=.3, scale="area")
plt.show()

plt.show()
plt.figure()
fig1 = seaborn.violinplot(y='intercept', x='x', data=df2, orient='v',  hue='mode', \
                          inner='stick', bw=.3, scale="area")
plt.show()

print (df2)
print (df2['sort'].value_counts())
#plt.ylim(-0.05, 0.05)

In [ ]:
import matplotlib.pyplot as plt

plt.figure()
seaborn.lmplot(x="comb", y="p_value", data=df6, hue='mode', fit_reg=False)
plt.savefig(out_dir + Title + "yvp.png")
plt.show()

### Define all tham thar functions

In [2]:
import pandas as pd
import seaborn
import pandas
import glob
import numpy
from scipy import stats
import re
import matplotlib.pyplot as plt

def fetch_var(samples):
    # Create dataframe for lin-reg stats
    df3 = pd.DataFrame(columns=[ 'Title', 'gene', 'slope','intercept', 'r_value', 'p_value', 'std_err', 'y_value', 'comb', 'sort'])
    data_dir ='/home/archie/code/DnA/'
   # for sample in samples:
    for filepath in glob.iglob(data_dir + samples +  "/HO*.tsv"):
      # Parse
        fields = ['non-promoterbi_MD', 'max_exp_of_TF', 'numtotalbi', 'SRR', 'max_exp_isoform' ]
        df = pd.read_csv(filepath, sep='\t', usecols=fields)
        np = 0
        nm = 0
        mean_MD = round(df['non-promoterbi_MD'].mean(), 5)
        for i in range(0, (len(df['non-promoterbi_MD']))):
            if (df.loc[i, 'numtotalbi'] < 5000):
                df = df.drop([i])
            try: 
                if (df.loc[i, 'non-promoterbi_MD'] > 0.1):
                    np +=1
                elif (df.loc[i, 'non-promoterbi_MD'] < 0.1):
                    nm +=1
            except:
                print ("")
        pop = ((len(df['non-promoterbi_MD']))*0.75)
        if np >= pop or mean_MD >= 0.115:
            sort = 'promoter'
        elif nm >= pop or mean_MD <= 0.08:
            sort ='repressor'
        else:
            sort = 'uncertain'

      # Get Gene Title
        regexp = "\/(?:HO_)(.*_.*)\."
        Title = re.findall(regexp, filepath)[0]
        regexp1 = "\/(?:HO_)(.*)(?:_.*)\." 
        regexp2 = "\/(?:HO_)(?:.*)(?:_.*)(.[A-Z])\."
        r1 = re.findall(regexp1, filepath)[0]
        r2 = re.findall(regexp2, filepath)[0]
        Gene = (r1 +  r2)

       # Generate lin-reg with sci-py
        slope, intercept, r_value, p_value, std_err = stats.linregress(df['max_exp_of_TF'], df['non-promoterbi_MD'])
        y_value = df['non-promoterbi_MD'].corr(df['max_exp_of_TF'], method='spearman')
      #  print (Title + "\nslope : " + str(slope) + "\nintercept : " + str(intercept) + "\nr_value : " + str(r_value) + "\np_value : " \
      #         + str(p_value) + "\nstd_err : " + str(std_err))
        print (Gene)
        comb = ((y_value + r_value)/2)
        df3.loc[Title] = [ Title, Gene, slope, intercept, r_value, p_value, std_err, y_value, comb, sort]
    df3['x'] = ''
    return (df3)
    print ("done")
   # print (df2)

print ("done")

def import_GC(Dataframe):
    GCMEANS_PATH = (r'/home/archie/code/DnA/data/metadata/meansGC.csv')
 #   GCMEANS="potato"
    GCMeans = pandas.read_csv(GCMEANS_PATH, sep=',')
    print (GCMeans)
    #GCMeans.columns = ['Title', 'GC', 'mean_MD', 'mean_lfpkm', 'std_lfpkm', 'co_var_lfpkm'\
               #       , 'spearman', 'pearson', 'kendall']
    df3 = pandas.merge(GCMeans, Dataframe, on='Title', how='outer')
    return df3
    print ("done")

def binnybinbin2(Dataframe):
    # 2x ways to sort
    #1). using average y and r
    for index, row in Dataframe.iterrows():
        if row['comb'] > 0.1 \
        and row['p_value'] < 0.05:
            if row['sort'] == 'promoter': 
                Dataframe.at[index, 'binvf'] = "positive_reg-promoter"
            elif row['sort'] == 'repressor': 
                Dataframe.at[index, 'binvf'] = "positive_reg-repressor"
            elif  row['sort'] == 'uncertain': 
                Dataframe.at[index, 'binvf'] = "positive_reg-uncertain"
            else: 
                Dataframe.at[index, 'binvf'] = "ERROR"  
        elif row['comb'] < -0.16 \
        and row['p_value'] < 0.05:
            if row['sort'] == 'promoter': 
                Dataframe.at[index, 'binvf'] = "negative_reg-promoter"
            elif row['sort'] == 'repressor': 
                Dataframe.at[index, 'binvf'] = "negative_reg-repressor"
            elif  row['sort'] == 'uncertain': 
                Dataframe.at[index, 'binvf'] = "negative_reg-uncertain"
            else: 
                Dataframe.at[index, 'binvf'] = "ERROR"  
        else:
            if row['sort'] == 'promoter':
                Dataframe.at[index, 'binvf'] = "unregulated-promoter"
            elif row['sort'] == 'repressor':
                Dataframe.at[index, 'binvf'] = "unregulated-repressor"
            elif row['sort'] == 'uncertain':
                Dataframe.at[index, 'binvf'] = "unregulated-uncertain"
            else: 
                Dataframe.at[index, 'binvf'] = "ERROR"  
    return (Dataframe)

print ("done")

def binnybinbin(Dataframe):
    # 2x ways to sort
    #1). using average y and r
    for index, row in Dataframe.iterrows():
        if row['comb'] > 0.1 \
        and row['p_value'] < 0.05:
            Dataframe.at[index, 'binv1'] = "T-reg-promoter"
            Dataframe.at[index, 'binv1n'] = int(3)
        elif row['comb'] < -0.16 \
        and row['p_value'] < 0.05:
            Dataframe.at[index, 'binv1'] = "T-reg-repressors"
            Dataframe.at[index, 'binv1n'] = int(1)
        else:
            Dataframe.at[index, 'binv1'] = "neither"
            Dataframe.at[index, 'binv1n'] = int(2)
    #2). using all 3 variables
    for index,row in Dataframe.iterrows():
        if row['r_value'] > 0.1  \
        and row['p_value'] < 0.05  \
        and row['y_value'] > 0.1:
            Dataframe.at[index, 'binv2'] = "T-reg-promoter"
            Dataframe.at[index, 'binv2n'] = 3
        elif row['r_value'] < -0.2 \
        and row['p_value'] < 0.05 \
        and row['y_value'] < -0.2:
            Dataframe.at[index, 'binv2'] = "T-reg-repressors"
            Dataframe.at[index, 'binv2n'] = int(1)
        else:
            Dataframe.at[index, 'binv2'] = "neither"
            Dataframe.at[index, 'binv2n'] = int(2)
    return (Dataframe)

print ("done")

def binnybinbin3(Dataframe):
    cutoff1 = .35
    for index, row in Dataframe.iterrows():
        print (row['binvf'])
        if row['binvf'] ==  "positive_reg-promoter": 
                if row['GC'] > cutoff1:
                      Dataframe.at[index, 'binvfGC1'] = "positive_reg-promoterGC+"
                elif row['GC'] < cutoff1:
                      Dataframe.at[index, 'binvfGC1'] = "positive_reg-promoterGC-"
        elif row['binvf'] ==  "negative_reg-promoter": 
                if row['GC'] > cutoff1:
                      Dataframe.at[index, 'binvfGC1'] = "negative_reg-promoterGC+"
                elif row['GC'] < cutoff1:
                      Dataframe.at[index, 'binvfGC1'] = "negative_reg-promoterGC-"  
        elif row['binvf'] ==  "unregulated-promoter": 
                if row['GC'] > cutoff1:
                      Dataframe.at[index, 'binvfGC1'] = "unregulated-promoterGC+"
                elif row['GC'] < cutoff1:
                      Dataframe.at[index, 'binvfGC1'] = "unregulated-promoterGC-"  
        elif row['binvf'] ==  "positive_reg-repressor": 
                if row['GC'] > cutoff1:
                      Dataframe.at[index, 'binvfGC1'] = "positive_reg-repressorGC+"
                elif row['GC'] < cutoff1:
                      Dataframe.at[index, 'binvfGC1'] = "positive_reg-repressorGC-"
        elif row['binvf'] ==  "negative_reg-repressor": 
                if row['GC'] > cutoff1:
                      Dataframe.at[index, 'binvfGC1'] = "negative_reg-repressorGC+"
                elif row['GC'] < cutoff1:
                      Dataframe.at[index, 'binvfGC1'] = "negative_reg-repressorGC-"  
        elif row['binvf'] ==  "unregulated-repressor": 
                if row['GC'] > cutoff1:
                      Dataframe.at[index, 'binvfGC1'] = "unregulated-repressorGC+"
                elif row['GC'] < cutoff1:
                      Dataframe.at[index, 'binvfGC1'] = "unregulated-repressorGC-"  
        elif row['binvf'] ==  "positive_reg-uncertain": 
                if row['GC'] > cutoff1:
                      Dataframe.at[index, 'binvfGC1'] = "positive_reg-uncertainGC+"
                elif row['GC'] < cutoff1:
                      Dataframe.at[index, 'binvfGC1'] = "positive_reg-uncertainGC-"
        elif row['binvf'] ==  "negative_reg-uncertain": 
                if row['GC'] > cutoff1:
                      Dataframe.at[index, 'binvfGC1'] = "negative_reg-uncertainGC+"
                elif row['GC'] < cutoff1:
                      Dataframe.at[index, 'binvfGC1'] = "negative_reg-uncertainGC-"  
        elif row['binvf'] == "unregulated-uncertain": 
                if row['GC'] > cutoff1:
                      Dataframe.at[index, 'binvfGC1'] = "unregulated-uncertainGC+"
                elif row['GC'] < cutoff1:
                      Dataframe.at[index, 'binvfGC1'] = "unregulated-uncertainGC-"  
        else:
              Dataframe.at[index, 'binvfGC1'] = "ERROR"
            
    return (df6)
    print ("done")
    
def bin2(df):
    df.reset_index()
    cutoff1 = 0.35
    cutoff2 = .53
    for i in range(0, 635):
        if df['GC'][i] < cutoff1:
          #  print (df['GC'][i])
            df.at[i, 'model'] = "GC<.35"
        elif df['GC'][i] > cutoff1 and df['GC'][i] < cutoff2:
            df.at[i, 'model'] = '.35<GC>.53'
        else:
            df.at[i, 'model'] = "GC>.53"
    cutoff1 = 0.35
    for i in range(0, 635):
        if df['GC'][i] < cutoff1:
          #  print (df['GC'][i])
            df.at[i, 'mode'] = "GC<.35"
        else:
            df.at[i, 'mode'] = "GC>.35"
    return(df)


done
done
done


### Calling all functions
#### calculated p, y, r, based on lin-regs of each motif
#### sort motifs by promoter/repressor and positive/negative/uncertain regulation
#### bin each motif according to GC content

In [5]:
## putting it all together

df3=fetch_var('data')

# GC text file here
GCpath = '/home/archie/code/DnA/data/metadata/gcrichdic.txt'

#parse +  combine
GC = pandas.read_csv(GCpath, sep='\t')
GC.columns = ['Title', 'GC']
df4 = pandas.merge(GC, df3, on='Title', how='inner')

df5 = (binnybinbin2(df4))
df6 = bin2(df5)
df7 = binnybinbin3(df6)

print (df7)









SCRT1.D







TBX3.D







ETV2.D







NFAT5.D







NANOG.S







HMX3.D







AP2D.D







ZIC3.C







KLF4.A







PLAL1.D







RARA.S







WT1.D







ZBTB4.S







FEV.C







LHX8.D







BHE40.A







ZN740.D







CDX2.C







ALX3.D







UBIP1.D







GSX2.D







TGIF2.D







MLX.D







SOX11.D







CLOCK.D







PEBB.C







BARX1.D







HXB7.C







CXXC1.D







AP2B.B







UNC4.D







NR1I2.S







ESR2.A







HES7.D







LBX2.D







PPARA.S







GSX1.D







FUBP1.D







EVI1.B







NFIC.A







BHE23.D







MAFB.D







FOXO3.B







PTF1A.C







NR1I3.S







ATOH1.D







HAND1.D







NKX61.D







E2F6.C







E2F3.B







KLF15.D







RFX2.C







RFX3.B







HXD9.D







STAT4.D







LMX1A.D







E2F2.B







FLI1.A







RUNX3.C







SOX2.B







TBP.C







PURA.D







TWST1.D







HXB2.D







MAFK.A







NFIA.S







NFIA.C







CREB5.D







GABPA.A







H








TF65.A







TBX4.D







PRRX2.C







HME1.D







STA5B.C







ZBTB6.D







ZKSC3.D







GCM1.D







PITX2.D







CENPB.D







INSM1.C







KLF1.C







ZN219.D







HEY2.D







PBX3.B







HXA2.D







NFAC3.B







GLI3.B







MZF1.D







USF1.A







EHF.S







PKNX1.D







REL.C







STAT2.B







HXA1.C







PO4F1.D







SPZ1.D







GCM2.D







NGN2.D







GSC.D







ZSCA4.D







NR2F6.D







HLTF.D







NKX31.C







HBP1.D







RUNX1.A







BATF.S







KAISO.S







MEIS2.B







ZBT49.D







ZBT18.D







TYY2.D







TEF.D







SOX13.D







CDC5L.D







PO4F2.D







OTX1.D







SP1.C







SPI1.A







PO3F2.D







HTF4.B







IKZF1.C







NFE2.B







HXB13.D







HINFP.C







BCL6.C







CR3L1.D







SMAD3.C







NR5A2.C







PRGR.S







ERR1.D







TFE3.C







NFAC2.B







PO3F4.D







HSF1.A







JDP2.D







PLAG1.D







TFE2.C







NF2L2.D
unreg

# EVERYTHING BELOW THIS POINT IS WORKSPACE

In [ ]:

def import_GC(Dataframe):
    GCMEANS_PATH = (r'/home/archie/code/DnA/data/metadata/meansGC.csv')
 #   GCMEANS="potato"
    GCMeans = pandas.read_csv(GCMEANS_PATH, sep=',')
    print (GCMeans)
    #GCMeans.columns = ['Title', 'GC', 'mean_MD', 'mean_lfpkm', 'std_lfpkm', 'co_var_lfpkm'\
               #       , 'spearman', 'pearson', 'kendall']
    df3 = pandas.merge(GCMeans, Dataframe, on='Title', how='outer')
    return df3
    print ("done")

In [ ]:


def binnybinbin(Dataframe):
    # 2x ways to sort
    #1). using average y and r
    for index, row in Dataframe.iterrows():
        if row['comb'] > 0.1 \
        and row['p_value'] < 0.05:
            Dataframe.at[index, 'binv1'] = "T-reg-promoter"
            Dataframe.at[index, 'binv1n'] = int(3)
        elif row['comb'] < -0.16 \
        and row['p_value'] < 0.05:
            Dataframe.at[index, 'binv1'] = "T-reg-repressors"
            Dataframe.at[index, 'binv1n'] = int(1)
        else:
            Dataframe.at[index, 'binv1'] = "neither"
            Dataframe.at[index, 'binv1n'] = int(2)
    #2). using all 3 variables
    for index,row in Dataframe.iterrows():
        if row['r_value'] > 0.1  \
        and row['p_value'] < 0.05  \
        and row['y_value'] > 0.1:
            Dataframe.at[index, 'binv2'] = "T-reg-promoter"
            Dataframe.at[index, 'binv2n'] = 3
        elif row['r_value'] < -0.2 \
        and row['p_value'] < 0.05 \
        and row['y_value'] < -0.2:
            Dataframe.at[index, 'binv2'] = "T-reg-repressors"
            Dataframe.at[index, 'binv2n'] = int(1)
        else:
            Dataframe.at[index, 'binv2'] = "neither"
            Dataframe.at[index, 'binv2n'] = int(2)
    return (Dataframe)

print ("done")

In [ ]:


def binnybinbin2(Dataframe):
    # 2x ways to sort
    #1). using average y and r
    for index, row in Dataframe.iterrows():
        if row['comb'] > 0.1 \
        and row['p_value'] < 0.05:
            if row['mean_MD'] > .11: 
                Dataframe.at[index, 'binvf'] = "positive_reg-promoter"
            elif row['mean_MD'] < 0.09: 
                Dataframe.at[index, 'binvf'] = "positive_reg-repressor"
            else:
                Dataframe.at[index, 'binvf'] = "positive_reg-uncertain"
        elif row['comb'] < -0.2 \
        and row['p_value'] < 0.05:
            if row['mean_MD'] > .11: 
                Dataframe.at[index, 'binvf'] = "negative_reg-promoter"
            elif row['mean_MD'] < 0.09: 
                Dataframe.at[index, 'binvf'] = "negative_reg-repressor"
            else:
                Dataframe.at[index, 'binvf'] = "negative_reg-uncertain"
        else:
            if row['mean_MD'] > .11: 
                Dataframe.at[index, 'binvf'] = "unregulated-promoter"
            elif row['mean_MD'] < 0.09: 
                Dataframe.at[index, 'binvf'] = "unregulated-repressor"
            else:
                Dataframe.at[index, 'binvf'] = "unregulated-uncertain"
    return (Dataframe)

def binnybinbin2(Dataframe):
    # 2x ways to sort
    #1). using average y and r
    for index, row in Dataframe.iterrows():
        if row['comb'] > 0.1 \
        and row['p_value'] < 0.05:
            if row['sort'] == 'promoter': 
                Dataframe.at[index, 'binvf'] = "positive_reg-promoter"
            elif row['sort'] == 'repressor': 
                Dataframe.at[index, 'binvf'] = "positive_reg-repressor"
            elif  row['sort'] == 'uncertain': 
                Dataframe.at[index, 'binvf'] = "positive_reg-uncertain"
            else: 
                Dataframe.at[index, 'binvf'] = "ERROR"  
        elif row['comb'] < -0.16 \
        and row['p_value'] < 0.05:
            if row['sort'] == 'promoter': 
                Dataframe.at[index, 'binvf'] = "negative_reg-promoter"
            elif row['sort'] == 'repressor': 
                Dataframe.at[index, 'binvf'] = "negative_reg-repressor"
            elif  row['sort'] == 'uncertain': 
                Dataframe.at[index, 'binvf'] = "negative_reg-uncertain"
            else: 
                Dataframe.at[index, 'binvf'] = "ERROR"  
        else:
            if row['sort'] == 'promoter':
                Dataframe.at[index, 'binvf'] = "unregulated-promoter"
            elif row['sort'] == 'repressor':
                Dataframe.at[index, 'binvf'] = "unregulated-repressor"
            elif row['sort'] == 'uncertain':
                Dataframe.at[index, 'binvf'] = "unregulated-uncertain"
            else: 
                Dataframe.at[index, 'binvf'] = "ERROR"  
    return (Dataframe)

print ("done")

In [ ]:


def binnybinbin3(Dataframe):
    cutoff1 = .35
    for index, row in Dataframe.iterrows():
        print (row['binvf'])
        if row['binvf'] ==  "positive_reg-promoter": 
                if row['GC'] > cutoff1:
                      Dataframe.at[index, 'binvfGC1'] = "positive_reg-promoterGC+"
                elif row['GC'] < cutoff1:
                      Dataframe.at[index, 'binvfGC1'] = "positive_reg-promoterGC-"
        elif row['binvf'] ==  "negative_reg-promoter": 
                if row['GC'] > cutoff1:
                      Dataframe.at[index, 'binvfGC1'] = "negative_reg-promoterGC+"
                elif row['GC'] < cutoff1:
                      Dataframe.at[index, 'binvfGC1'] = "negative_reg-promoterGC-"  
        elif row['binvf'] ==  "unregulated-promoter": 
                if row['GC'] > cutoff1:
                      Dataframe.at[index, 'binvfGC1'] = "unregulated-promoterGC+"
                elif row['GC'] < cutoff1:
                      Dataframe.at[index, 'binvfGC1'] = "unregulated-promoterGC-"  
        elif row['binvf'] ==  "positive_reg-repressor": 
                if row['GC'] > cutoff1:
                      Dataframe.at[index, 'binvfGC1'] = "positive_reg-repressorGC+"
                elif row['GC'] < cutoff1:
                      Dataframe.at[index, 'binvfGC1'] = "positive_reg-repressorGC-"
        elif row['binvf'] ==  "negative_reg-repressor": 
                if row['GC'] > cutoff1:
                      Dataframe.at[index, 'binvfGC1'] = "negative_reg-repressorGC+"
                elif row['GC'] < cutoff1:
                      Dataframe.at[index, 'binvfGC1'] = "negative_reg-repressorGC-"  
        elif row['binvf'] ==  "unregulated-repressor": 
                if row['GC'] > cutoff1:
                      Dataframe.at[index, 'binvfGC1'] = "unregulated-repressorGC+"
                elif row['GC'] < cutoff1:
                      Dataframe.at[index, 'binvfGC1'] = "unregulated-repressorGC-"  
        elif row['binvf'] ==  "positive_reg-uncertain": 
                if row['GC'] > cutoff1:
                      Dataframe.at[index, 'binvfGC1'] = "positive_reg-uncertainGC+"
                elif row['GC'] < cutoff1:
                      Dataframe.at[index, 'binvfGC1'] = "positive_reg-uncertainGC-"
        elif row['binvf'] ==  "negative_reg-uncertain": 
                if row['GC'] > cutoff1:
                      Dataframe.at[index, 'binvfGC1'] = "negative_reg-uncertainGC+"
                elif row['GC'] < cutoff1:
                      Dataframe.at[index, 'binvfGC1'] = "negative_reg-uncertainGC-"  
        elif row['binvf'] == "unregulated-uncertain": 
                if row['GC'] > cutoff1:
                      Dataframe.at[index, 'binvfGC1'] = "unregulated-uncertainGC+"
                elif row['GC'] < cutoff1:
                      Dataframe.at[index, 'binvfGC1'] = "unregulated-uncertainGC-"  
        else:
              Dataframe.at[index, 'binvfGC1'] = "ERROR"
            
    return (df6)
    print ("done")
    
#print (df6)



In [ ]:
df6 = binnybinbin2(df6)

In [ ]:
print (df6)

In [ ]:
def bin2(df):
    df.reset_index()
    cutoff1 = 0.35
    cutoff2 = .53
    for i in range(0, 635):
        if df['GC'][i] < cutoff1:
          #  print (df['GC'][i])
            df.at[i, 'model'] = "GC<.35"
        elif df['GC'][i] > cutoff1 and df['GC'][i] < cutoff2:
            df.at[i, 'model'] = '.35<GC>.53'
        else:
            df.at[i, 'model'] = "GC>.53"
    cutoff1 = 0.35
    for i in range(0, 635):
        if df['GC'][i] < cutoff1:
          #  print (df['GC'][i])
            df.at[i, 'mode'] = "GC<.35"
        else:
            df.at[i, 'mode'] = "GC>.35"
    return(df)



In [ ]:
# Enter save dir here
out_dir = '/home/archie/code/DnA/out/presy2/'print (df4)

In [ ]:

# Enter save dir here
out_dir = '/home/archie/code/DnA/out/presy2/'

plt.figure()
fig1 = seaborn.violinplot(y='intercept', x='x', data=df3, orient='v', split=True, \
                          inner='stick', bw=.3, scale="area")
plt.ylim(0.05,0.25)
plt.savefig(out_dir + "interceptavg")
plt.show()

plt.figure()
fig1 = seaborn.violinplot(y='mean_MD', x='x', data=df3, orient='v', split=True, \
                          inner='stick', bw=.3, scale="area")
plt.ylim(0.05,0.25)
plt.savefig(out_dir + "meanMDvp")
plt.show()



plt.figure()
fig1 = seaborn.lmplot(y='comb', x='p_value', data=df3, fit_reg=False)
plt.savefig(out_dir + "combvslopetotal.png")
plt.show()

In [ ]:
# Enter save dir here
out_dir = '/home/archie/code/DnA/out/presy2/'

#seaborn.lmplot(y="r_value", x="p_value", data=df2, hue=mode,  fit_reg=False)
plt.figure()
fig1 = seaborn.violinplot(y='p_value', x='x', data=df3, orient='v',# split=True, \
                          inner='stick', bw=.3, scale="area")
plt.show()
plt.figure()
fig1 = seaborn.violinplot(y='r_value', x='x', data=df3, orient='v', #split=True, \
                          inner='stick', bw=.3, scale="area")
plt.show()
plt.figure()
fig1 = seaborn.violinplot(y='slope', x='x', data=df3, orient='v', split=True, \
                          inner='stick', bw=.3, scale="area")
plt.show()
plt.figure()
fig1 = seaborn.violinplot(y='intercept', x='x', data=df3, orient='v', split=True, \
                          inner='stick', bw=.3, scale="area")
plt.ylim(0.05,0.25)
plt.savefig(out_dir + "interceptavg")
plt.show()
plt.figure()
fig1 = seaborn.lmplot(y='comb', x='p_value', data=df3, fit_reg=False)
plt.savefig(out_dir + "combvslopetotal.png")
plt.show()

print ("done")

In [ ]:
### combined plots

In [ ]:
plt.figure()
seaborn.lmplot(y="r_value", x="p_value", data=df3, fit_reg=False)
plt.show()
plt.figure()
seaborn.lmplot(y="y_value", x="p_value", data=df3, fit_reg=False)
plt.show()
plt.figure()
seaborn.lmplot(y="comb", x="p_value", data=df3, fit_reg=False)
plt.show()

In [5]:
## putting it all together

df3=fetch_var('data')

# GC text file here
GCpath = '/home/archie/code/DnA/data/metadata/gcrichdic.txt'

#parse +  combine
GC = pandas.read_csv(GCpath, sep='\t')
GC.columns = ['Title', 'GC']
df4 = pandas.merge(GC, df3, on='Title', how='inner')

df5 = (binnybinbin2(df4))
df6 = bin2(df5)
df7 = binnybinbin3(df6)

print (df7)









SCRT1.D







TBX3.D







ETV2.D







NFAT5.D







NANOG.S







HMX3.D







AP2D.D







ZIC3.C







KLF4.A







PLAL1.D







RARA.S







WT1.D







ZBTB4.S







FEV.C







LHX8.D







BHE40.A







ZN740.D







CDX2.C







ALX3.D







UBIP1.D







GSX2.D







TGIF2.D







MLX.D







SOX11.D







CLOCK.D







PEBB.C







BARX1.D







HXB7.C







CXXC1.D







AP2B.B







UNC4.D







NR1I2.S







ESR2.A







HES7.D







LBX2.D







PPARA.S







GSX1.D







FUBP1.D







EVI1.B







NFIC.A







BHE23.D







MAFB.D







FOXO3.B







PTF1A.C







NR1I3.S







ATOH1.D







HAND1.D







NKX61.D







E2F6.C







E2F3.B







KLF15.D







RFX2.C







RFX3.B







HXD9.D







STAT4.D







LMX1A.D







E2F2.B







FLI1.A







RUNX3.C







SOX2.B







TBP.C







PURA.D







TWST1.D







HXB2.D







MAFK.A







NFIA.S







NFIA.C







CREB5.D







GABPA.A







H








TF65.A







TBX4.D







PRRX2.C







HME1.D







STA5B.C







ZBTB6.D







ZKSC3.D







GCM1.D







PITX2.D







CENPB.D







INSM1.C







KLF1.C







ZN219.D







HEY2.D







PBX3.B







HXA2.D







NFAC3.B







GLI3.B







MZF1.D







USF1.A







EHF.S







PKNX1.D







REL.C







STAT2.B







HXA1.C







PO4F1.D







SPZ1.D







GCM2.D







NGN2.D







GSC.D







ZSCA4.D







NR2F6.D







HLTF.D







NKX31.C







HBP1.D







RUNX1.A







BATF.S







KAISO.S







MEIS2.B







ZBT49.D







ZBT18.D







TYY2.D







TEF.D







SOX13.D







CDC5L.D







PO4F2.D







OTX1.D







SP1.C







SPI1.A







PO3F2.D







HTF4.B







IKZF1.C







NFE2.B







HXB13.D







HINFP.C







BCL6.C







CR3L1.D







SMAD3.C







NR5A2.C







PRGR.S







ERR1.D







TFE3.C







NFAC2.B







PO3F4.D







HSF1.A







JDP2.D







PLAG1.D







TFE2.C







NF2L2.D
unreg

In [ ]:
## putting it all together

#df3=fetch_var('data')

# GC text file here
GCpath = (r'/home/archie/code/DnA/data/metadata/meansGC.csv')

#parse +  combine
GC = pandas.read_csv(GCpath, sep=',')
#GC.columns = ['Title', 'GC', 'mean_MD', 'mean_lfpkm', 'std_lfpkm', 'co_var_lfpkm'\
#                      , 'spearman', 'pearson', 'kendall']

df4 = pandas.merge(GC, df3, on='Title', how='inner')

df5 = (binnybinbin2(df4))
df6 = bin2(df5)
print (df6)

df7 = binnybinbin3(df6)


In [ ]:
plt.figure()
fig1 = seaborn.lmplot(y='comb', x='p_value', data=df6, hue='mode', fit_reg=False)
plt.xlim(0,0.01)
plt.show()

plt.figure()
fig1 = seaborn.violinplot(y='comb', x='x', data=df6, hue='mode', split=True, scale="area", bw=0.2)
plt.show()

In [ ]:
#df7 = binnybinbin3(df6)

#print (df7)
print (df7['binvf'].value_counts())

In [ ]:
print (df7['binvfGC1'].value_counts())

In [ ]:
for i in range(0, len(df6)):
    if df6['binv2'][i] == 'T-reg-promoter':
        print (df6['Title'][i])

In [ ]:
print (df6[NAN])

In [ ]:
binnybinbin(df3)
print (df6['binv1'].value_counts())
print (df4['binv1n'].value_counts())
print (df4['binv2'].value_counts())
print (df4['binv2n'].value_counts())

In [ ]:
df4['quartiles'] = pd.qcut(df4['GC'], 10, duplicates='drop')        
#df4['x'] = ''
plt.figure()
fig1 = seaborn.violinplot(y='binv1n', x='quartiles', data=df4, inner='stick', orient='v', bw=.2, scale="area")
plt.show()

#df4['x'] = ''
plt.figure()
fig1 = seaborn.violinplot(y='GC', x='binv1', data=df4, inner='stick', orient='v', bw=.2, scale="area")
plt.show()

In [ ]:
plt.figure()
fig1 = seaborn.violinplot(y='binv1n', x='mode', data=df6, inner='stick', orient='v', bw=.2, scale="area")
plt.show()
plt.figure()
fig1 = seaborn.violinplot(y='binv1n', x='model', data=df6, inner='stick', orient='v', bw=.2, scale="area")
plt.show()